In [1]:
import pandas as pd
from textblob import TextBlob

In [2]:
pd.set_option('display.max_colwidth', -1)

/Users/a0s01w2/Downloads/anaconda3/envs/absa/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [4]:
data = pd.read_csv('Case_data_processed.csv')

## Basic TextBlob Approach

In [16]:
#https://github.com/iamvishnuks/ABSA/blob/master/absa.py

def get_aspect_sentiment(text):
    aspect_sentiment = []
    blob=TextBlob(text)
    tokens=list(blob.words)
    word=[]
    sent=[]
    c=0
    i=0
    for words,pos in blob.tags:
        if pos=='JJ' or pos=='NN' or pos=='JJR' or pos=='NNS':
            word.append(words)
    if len(word)>=2:
            firstw=word[0]
            secw=word[1]
            word.remove(firstw)
            word.remove(secw)
            findx=tokens.index(firstw)
            lindx=tokens.index(secw)
            sent.append(' '.join(tokens[findx:lindx+1]))
   
    for sentence in sent:
        aspect_sentiment.append((sentence, TextBlob(sentence).polarity))
        
    return aspect_sentiment

In [17]:
print(data['Description'].iloc[1])
get_aspect_sentiment(data['Description'].iloc[1])

I am new and trying to upload my items.  I noticed that there is an error appearing each time that I add a new item.  It is the same error each time.    This is what appears: SKU	Error Type	Error(s) This is the SKU that has an error in your feed	The type of error associated with your item setup failure.	If you have more than 1 error, they will appear in new columns to the right for a given SKU 503-FT	Data error	ID Type=GTIN, ID Value=856453005716, invalid product ID length: 12


[('new and trying to upload my items', 0.13636363636363635)]

In [18]:
print(data['Description'].iloc[2])
get_aspect_sentiment(data['Description'].iloc[2])

I want to  know how to creat a item listing with many variation?  i dont want to use bulk , but i cant find the way to add variation in single item now.


[('item listing with many', 0.5)]

In [19]:
print(data['Description'].iloc[3])
get_aspect_sentiment(data['Description'].iloc[3])

In this account, all online selling products are home textiles. Can I sell other category products, such as clothing products and electronic products？


[('account all online selling', 0.0)]

## TextBlob Approach 2

https://github.com/OrthoDex/ABSA-YouTube/blob/master/ABSA%20of%20YouTube%20Comments.ipynb

In [22]:
%matplotlib inline
from textblob import TextBlob, Word
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

from string import punctuation
import re

In [8]:
# Text cleaning 

def clean_sentence(sentence):
    sentence = re.sub(r"(?:\@|https?\://)\S+|\n+", "", sentence.lower())
    # Fix spelling errors in comments!
    sent = TextBlob(sentence)
    sent.correct()
    clean = ""
    for sentence in sent.sentences:    
        words = sentence.words
        # Remove punctuations
        words = [''.join(c for c in s if c not in punctuation) for s in words]
        words = [s for s in words if s]
        clean += " ".join(words)
        clean += ". "
    return clean

In [9]:
def compactness_pruning(text):
    comments = TextBlob(text)
    #compactness pruning:
    cleaned = list()
    for phrase in comments.noun_phrases:
        count = 0
        for word in phrase.split():
            # Count the number of small words and words without an English definition
            if len(word) <= 2 or (not Word(word).definitions):
                count += 1
        # Only if the 'nonsensical' or short words DO NOT make up more than 40% (arbitrary) of the phrase add
        # it to the cleaned list, effectively pruning the ones not added.
        if count < len(phrase.split())*0.4:
            cleaned.append(phrase)
            
    return cleaned

In [10]:
text = data_subset['Description'].iloc[4]

In [12]:
text_clean = clean_sentence(text)

In [16]:
noun_phrases = compactness_pruning(text_clean)

In [22]:
absa_dict = dict()
blob = TextBlob(text)
for f in noun_phrases:
    absa_dict[f] = list()
    # For each sentence of the comment
    for sentence in blob.sentences:
        # Search for frequent feature 'f'
        q = '|'.join(f.split())
        if re.search(r'\w*(' + str(q) + ')\w*', str(sentence)):
            absa_dict[f].append(sentence)

In [24]:
scores = list()
absa_scores = dict()
for k, v in absa_dict.items():
    absa_scores[k] = list()
    for sent in v:
        score = sent.sentiment.polarity
        scores.append(score)
        absa_scores[k].append(score)

In [28]:
absa_scores

{'multiple support representatives': [0.0]}